In [1]:
import os

import h5py
import numpy as np
import simplstyles
import matplotlib.pyplot as plt
import seaborn as sns
import voxelwise_tutorials.viz as viz
from matplotlib import pyplot as plt
from scipy.stats import zscore
from voxelwise_tutorials.io import load_hdf5_array

In [2]:
def get_path(language_model, feature, modality, subject, low_level_feature, layer):
    path = os.path.join("results", language_model, feature, modality, f"subject{subject:02}", low_level_feature,
                        f"layer{layer}")
    return path

In [3]:
plt.style.use('nord-light-talk')
data_dir = "../data"

In [4]:
language_model = "bert-base"
feature = "semantic"
modality = "reading"
subject = 1
layer = 9
low_level_feature = "letters"
trim = 5  # remove 5 TRs from the start and end of each story

# Load Features

## Load Low Level Feature

In [5]:
low_level_train = h5py.File(os.path.join(data_dir, 'features', 'features_trn_NEW.hdf'), 'r')
low_level_val = h5py.File(os.path.join(data_dir, 'features', 'features_val_NEW.hdf'), 'r')
print(low_level_train.keys())
print(low_level_val.keys())

<KeysViewHDF5 ['story_01', 'story_02', 'story_03', 'story_04', 'story_05', 'story_06', 'story_07', 'story_08', 'story_09', 'story_10']>
<KeysViewHDF5 ['story_11']>


In [6]:
low_level_train = np.vstack(
    [zscore(low_level_train[story][low_level_feature][trim:-trim]) for story in low_level_train.keys()])
low_level_val = np.vstack(
    [zscore(low_level_val[story][low_level_feature][trim:-trim]) for story in low_level_val.keys()])
low_level_train, low_level_val = np.nan_to_num(low_level_train), np.nan_to_num(low_level_val)
low_level_train

array([[-0.4700195 , -0.73438869,  0.4915585 , ..., -0.1790036 ,
         0.34119878, -0.11434134],
       [ 0.25931483,  1.02067184, -0.91108246, ..., -0.1790036 ,
        -0.2714388 , -0.11434134],
       [-1.62907001, -0.35271967, -0.80764183, ..., -0.1790036 ,
         0.33492298, -0.11434134],
       ...,
       [ 0.18454009, -0.5399342 , -0.66761462, ..., -0.28713784,
        -0.48108308, -0.13844843],
       [-1.25051674, -0.5676835 , -0.69646574, ..., -0.28713784,
         0.23124328, -0.13844843],
       [-1.28809956, -0.5676835 , -0.69672374, ..., -0.28713784,
        -1.19791374, -0.13844843]], shape=(3787, 26))

## Load High Level (NLP) Features

# Load Brain Data

In [7]:
Y_train_filename = os.path.join(data_dir, 'responses', f'subject{subject:02}_{modality}_fmri_data_trn.hdf')
Y_train = load_hdf5_array(Y_train_filename)

Y_test_filename = os.path.join(data_dir, 'responses', f'subject{subject:02}_{modality}_fmri_data_val.hdf')
Y_test = load_hdf5_array(Y_test_filename)

Y_train = np.vstack([zscore(Y_train[story][trim:-trim]) for story in Y_train.keys()])
Ys_test = [np.vstack([zscore(Y_test[story][i][trim:-trim]) for story in Y_test.keys()]) for i in range(2)]

/tmp/ipykernel_19542/591648698.py:7: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  Y_train = np.vstack([zscore(Y_train[story][trim:-trim]) for story in Y_train.keys()])
/tmp/ipykernel_19542/591648698.py:7: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  Y_train = np.vstack([zscore(Y_train[story][trim:-trim]) for story in Y_train.keys()])
/tmp/ipykernel_19542/591648698.py:7: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  Y_train = np.vstack([zscore(Y_train[story][trim:-trim]) for story in Y_train.keys()])
/tmp/ipykernel_19542/591648698.py:7: RuntimeWarning: Precision loss occurred in moment calculation due to catast

# Variance Partitioning

## Low Level Prediction

In [ ]:
Rstim, Pstim = load_z_low_level_feature(data_dir, low_level_feature)

# delay stimuli to account for hemodynamic lag
delays = range(1, number_of_delays + 1)
ct = ColumnTransformer([("low_level", Delayer(delays), slice(0, Rstim.shape[1] - 1))])

# fit bootstrapped ridge regression model
corrs, coef, alphas = bootstrap_ridge(Rstim, Rresp, Pstim, Presp, ct)

# save voxelwise correlations and predictions
output_file = get_prediction_path(language_model=None, feature="low-level", modality=modality, subject=subject_num,
                                  low_level_feature=low_level_feature)
output_dir = os.path.dirname(output_file)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
np.save(output_file, corrs)

# Residual Method

# Plot Brain Maps

In [8]:
def plot_flatmap_from_mapper(voxels, mapper_file, ax=None, alpha=0.7, cmap=plt.get_cmap(), vmin=None, vmax=None,
                             with_curvature=True, with_rois=True, with_colorbar=True,
                             colorbar_location=(.4, .9, .2, .05)):
    """Plot a flatmap from a mapper file, with 1D data.

    This function is equivalent to the pycortex functions:
    cortex.quickshow(cortex.Volume(voxels, ...), ...)

    Note that this function does not have the full capability of pycortex,
    since it is based on flatmap mappers and not on the original brain
    surface of the subject.

    Parameters
    ----------
    voxels : array of shape (n_voxels, )
        Data to be plotted.
    mapper_file : str
        File name of the mapper.
    ax : matplotlib Axes or None.
        Axes where the figure will be plotted.
        If None, a new figure is created.
    alpha : float in [0, 1], or array of shape (n_voxels, )
        Transparency of the flatmap.
    cmap : str
        Name of the matplotlib colormap.
    vmin : float or None
        Minimum value of the colormap. If None, use the 1st percentile of the
        `voxels` array.
    vmax : float or None
        Minimum value of the colormap. If None, use the 99th percentile of the
        `voxels` array.
    with_curvature : bool
        If True, show the curvature below the data layer.
    with_rois : bool
        If True, show the ROIs labels above the data layer.
    colorbar_location : [left, bottom, width, height]
        Location of the colorbar. All quantities are in fractions of figure
        width and height.

    Returns
    -------
    ax : matplotlib Axes
        Axes where the figure has been plotted.
    """
    # create a figure
    if ax is None:
        flatmap_mask = load_hdf5_array(mapper_file, key='flatmap_mask')
        figsize = np.array(flatmap_mask.shape) / 100.
        fig = plt.figure(figsize=figsize)
        ax = fig.add_axes((0, 0, 1, 1))
        ax.axis('off')

    # process plotting parameters
    vmin = np.percentile(voxels, 1) if vmin is None else vmin
    vmax = np.percentile(voxels, 99) if vmax is None else vmax
    if isinstance(alpha, np.ndarray):
        alpha = viz.map_voxels_to_flatmap(alpha, mapper_file)

    # plot the data
    image = viz.map_voxels_to_flatmap(voxels, mapper_file)
    cimg = ax.imshow(image, aspect='equal', zorder=1, alpha=alpha, cmap=cmap,
                     vmin=vmin, vmax=vmax)

    if with_colorbar:
        try:
            cbar = ax.inset_axes(colorbar_location)
        except AttributeError:  # for matplotlib < 3.0
            cbar = ax.figure.add_axes(colorbar_location)
        colorbar = ax.figure.colorbar(cimg, cax=cbar, orientation='horizontal')
        colorbar.ax.set_title("Pearson Correlation of Y_true and Y_pred", fontsize=14)

    # plot additional layers if present
    viz._plot_addition_layers(ax=ax, n_voxels=voxels.shape[0],
                              mapper_file=mapper_file,
                              with_curvature=with_curvature, with_rois=with_rois)

    return ax

In [10]:
path = get_path(language_model, feature, modality, subject, low_level_feature, layer)
correlation = np.nan_to_num(np.load(path, allow_pickle=True))
mapper_path = os.path.join("../data",'mappers', f"subject{subject:02}_mappers.hdf")
flatmap_mask = load_hdf5_array(mapper_path, key='flatmap_mask')

FileNotFoundError: [Errno 2] No such file or directory: 'results/bert-base/semantic/reading/subject01/letters/layer9'

In [ ]:
figsize = np.array(flatmap_mask.shape) / 100.
fig = plt.figure(figsize=figsize)
ax = fig.add_axes((0, 0, 1, 1))
ax.axis('off')
plot_flatmap_from_mapper(correlation, mapper_path, ax=ax, with_curvature=False, alpha=1, vmin=0,
                         # vmin=np.min(correlation),
                         vmax=np.max(correlation), colorbar_location=[0.75, 0.05, 0.2, 0.05])
plt.show()